# Módulo 5 Tarefa 1
## Base de nascidos vivos do DataSUS
O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet.). 

Essas informações podem ser obtidas pela internet [aqui](http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1). Como o processo de obtenção desses arquivos foge um pouco do nosso escopo, deixamos o arquivo SINASC_RO_2019.csv` já como vai ser encontrado no DataSUS. O dicionário de dados está no arquivo `estrutura_sinasc_para_CD.pdf` (o nome do arquivo tal qual no portal do DataSUS).

### Nosso objetivo
Queremos deixar uma base organizada para podermos estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.

#### Preparação da base
1. Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**  

2. Conte o número de valores *missing* por variável.  

3. Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
` 
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 
`
Refaça a contagem de valores *missings*.  

4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.  

5. observe que as variáveis `['ESTCIVMAE', 'CONSULTAS']` possuem o código `9`, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código `9`.<br>
6. Substitua os valores faltantes da quantitativa (`QTDFILVIVO`) por zero.  
7. Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.  
8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'. 
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'. 
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.  
<br>
9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção `inplace = True`.

In [1]:
import pandas as pd
import requests

# 1) seu código aqui
df = pd.read_csv('SINASC_RO_2019.csv')
print(df.shape)
df.drop_duplicates().shape
# Não há duplicados

(27028, 69)


(27028, 69)

In [2]:
# 2) seu código aqui
colunas_com_soma_maior_que_0 = df.loc[:, df.isna().sum() > 0]
colunas_com_soma_maior_que_0.columns
colunas_com_soma_maior_que_0.isna().sum().sort_values()

munResArea        1
munResLon         1
munResLat         1
munResAlt         1
SEXO              4
TPDOCRESP        14
HORANASC         21
DTNASCMAE        40
PARTO            49
DTDECLARAC       52
TPNASCASSI       61
TPFUNCRESP       67
GRAVIDEZ         79
APGAR1           96
APGAR5          103
CODESTAB        115
ESCMAE2010      249
ESCMAEAGR1      249
TPAPRESENT      265
NATURALMAE      298
CODMUNNATU      298
CODUFNATU       298
ESCMAE          312
ESTCIVMAE       317
IDANOMAL        591
RACACOR         647
RACACORMAE      661
STCESPARTO      747
CONSPRENAT      930
STTRABPART      947
QTDGESTANT     1212
GESTACAO       1232
SEMAGESTAC     1238
TPMETESTIM     1238
QTDFILVIVO     1573
QTDPARTCES     1791
QTDPARTNOR     1879
QTDFILMORT     2098
MESPRENAT      2867
CODOCUPMAE     2907
DTULTMENST    10072
SERIESCMAE    12710
IDADEPAI      19421
CODANOMAL     26814
DTRECORIGA    27028
dtype: int64

In [3]:
# 3) seu código aqui
dados_filtrados = df[['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO',
                  'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']]
dados_filtrados
dados_filtrados.isna().sum().sort_values()

LOCNASC          0
IDADEMAE         0
CONSULTAS        0
GRAVIDEZ        79
APGAR5         103
ESCMAE         312
ESTCIVMAE      317
GESTACAO      1232
QTDFILVIVO    1573
dtype: int64

In [4]:
# 4) seu código aqui 
df['APGAR5'].dropna()

0        10.0
1         9.0
2        10.0
3        10.0
4        10.0
         ... 
27023     9.0
27024     9.0
27025    10.0
27026     9.0
27027     8.0
Name: APGAR5, Length: 26925, dtype: float64

In [5]:
# 4) seu código aqui 
df = df.dropna(subset = 'APGAR5')

In [6]:
# 5) seu código aqui
df_sem_nove = df.loc[(df['CONSULTAS'] != 9) & (df['ESTCIVMAE'] != 9)]
df_sem_nove['CONSULTAS'].value_counts().sort_index()

CONSULTAS
1      966
2     1354
3     5925
4    18486
Name: count, dtype: int64

In [7]:
# 6) seu código aqui
df['QTDFILVIVO'].fillna(0,inplace= True)

In [8]:
# 7) seu código aqui
dados_filtrados = df[['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO',
                  'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']]
dados_filtrados
dados_filtrados.isna().sum().sort_values()

LOCNASC          0
IDADEMAE         0
QTDFILVIVO       0
CONSULTAS        0
APGAR5           0
GRAVIDEZ        76
ESCMAE         310
ESTCIVMAE      315
GESTACAO      1216
dtype: int64

In [9]:
(df['GRAVIDEZ'] != 'Única').sum()

569

In [10]:
df['ESCMAE']

0            8 a 11 anos
1            8 a 11 anos
2            8 a 11 anos
3        12 anos ou mais
4            8 a 11 anos
              ...       
27023    12 anos ou mais
27024        8 a 11 anos
27025        8 a 11 anos
27026        8 a 11 anos
27027        8 a 11 anos
Name: ESCMAE, Length: 26925, dtype: object

In [11]:
df['ESTCIVMAE']

0        5.0
1        2.0
2        9.0
3        5.0
4        2.0
        ... 
27023    2.0
27024    1.0
27025    2.0
27026    2.0
27027    5.0
Name: ESTCIVMAE, Length: 26925, dtype: float64

In [12]:
df['GESTACAO']

0        37 a 41 semanas
1        37 a 41 semanas
2        37 a 41 semanas
3        37 a 41 semanas
4        37 a 41 semanas
              ...       
27023    32 a 36 semanas
27024    37 a 41 semanas
27025    37 a 41 semanas
27026    32 a 36 semanas
27027    37 a 41 semanas
Name: GESTACAO, Length: 26925, dtype: object

In [13]:
# 7) seu código aqui
mode = df['GRAVIDEZ'].mode
mediana = df['ESTCIVMAE'].median()

df['GRAVIDEZ'].fillna(mode,inplace = True)
df['ESCMAE'].ffill(inplace = True)
df['ESTCIVMAE'].fillna(mediana,inplace = True)
df['GESTACAO'].ffill(inplace = True)

In [14]:
''# 8) seu código aqui
df.loc[df['APGAR5'] >= 8, 'apgar5_novo'] = 'normal'
df.loc[(df['APGAR5'] >=6) & (df['APGAR5'] <= 7),
           'apgar5_novo'] = 'asfixia leve'
df.loc[(df['APGAR5'] >= 4) & (df['APGAR5'] <= 5),
           'apgar5_novo'] = 'asfixia moderada'
df.loc[(df['APGAR5'] <= 3), 'apgar5_novo'] = 'asfixia severa'

In [15]:
df['apgar5_novo'].value_counts()

apgar5_novo
normal              26463
asfixia leve          320
asfixia severa         74
asfixia moderada       68
Name: count, dtype: int64

In [16]:
# 9) seu código aqui
df['apgar5_novo'] = df['apgar5_novo'].str.replace(' ', '_').str.lower()


In [17]:
df['apgar5_novo'].value_counts()

apgar5_novo
normal              26463
asfixia_leve          320
asfixia_severa         74
asfixia_moderada       68
Name: count, dtype: int64

In [18]:
df.to_csv('SINASC_RO_2019_.csv')